# Create a time-of-flight lookup table for DREAM

This notebook shows how to create a time-of-flight lookup table for the DREAM instrument.

In [ ]:
import scipp as sc
from ess.reduce import time_of_flight
from ess.reduce.nexus.types import AnyRun
from ess.dream.beamline import InstrumentConfiguration, choppers

## Select the choppers

We select the choppers for the 'high-flux' configuration.

In [ ]:
disk_choppers = choppers(InstrumentConfiguration.high_flux)

## Setting up the workflow

In [ ]:
wf = time_of_flight.TofLookupTableWorkflow()

wf[time_of_flight.LtotalRange] = sc.scalar(60.0, unit="m"), sc.scalar(80.0, unit="m")
wf[time_of_flight.NumberOfSimulatedNeutrons] = 200_000  # Increase this number for more reliable results
wf[time_of_flight.SourcePosition] = sc.vector([0, 0, 0], unit='m')
wf[time_of_flight.DiskChoppers[AnyRun]] = disk_choppers
wf[time_of_flight.DistanceResolution] = sc.scalar(0.1, unit="m")
wf[time_of_flight.TimeResolution] = sc.scalar(250.0, unit='us')
wf[time_of_flight.LookupTableRelativeErrorThreshold] = 0.02
wf[time_of_flight.PulsePeriod] = 1.0 / sc.scalar(14.0, unit="Hz")
wf[time_of_flight.PulseStride] = 1
wf[time_of_flight.PulseStrideOffset] = None

## Compute the table

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table

In [ ]:
table.plot()

## Save to file

In [ ]:
table.save_hdf5('DREAM-high-flux-tof-lookup-table.h5')